# Виправлення Помилки: CLIPTokenizer не підтримує метод `.to(device)`

У цьому ноутбуці ми виправимо помилку, пов'язану з використанням методу `.to(device)` на `CLIPTokenizer`. Токенізатори не потребують перенесення на пристрій, тому цей виклик призводить до помилки.

## Встановлення Необхідних Бібліотек

In [ ]:
# Встановлення необхідних бібліотек
!pip install --upgrade diffusers transformers torch torchvision matplotlib safetensors

## Імпорт Необхідних Модулів

In [ ]:
# Імпорт необхідних модулів
from diffusers import StableDiffusionXLPipeline
from transformers import CLIPTextModel, CLIPTokenizer
import torch
from PIL import Image
import matplotlib.pyplot as plt

## Перевірка Наявності GPU

In [ ]:
# Перевірка наявності GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Використовується пристрій: {device}")

## Завантаження Моделі для Генерації Зображень

In [ ]:
# Визначення ідентифікатора моделі
model_id = "Laxhar/noobai-XL-1.1"

# Ініціалізація пайплайна та перенесення на пристрій
try:
    pipe = StableDiffusionXLPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto"  # Автоматичне розподілення на доступні пристрої
    )
    pipe = pipe.to(device)
    print("Пайплайн успішно завантажено та перенесено на пристрій.")
except Exception as e:
    print(f"Не вдалося завантажити пайплайн: {e}")

## Ініціалізація Токенізатора та Текстових Енкодерів

In [ ]:
# Ініціалізація токенізатора та текстових енкодерів
try:
    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
    text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    text_encoder_2 = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    print("Токенізатор та текстові енкодери успішно завантажено.")
except Exception as e:
    print(f"Не вдалося завантажити токенізатор або текстові енкодери: {e}")

## Функція для Генерації Зображень

In [ ]:
def generate_image(prompt, guidance_scale=7.5, num_inference_steps=50):
    """
    Генерує зображення на основі текстового опису.
    
    Parameters:
    - prompt (str): Текстовий опис зображення.
    - guidance_scale (float): Контролює вплив текстового опису на зображення. Вищі значення роблять зображення більш відповідним до опису.
    - num_inference_steps (int): Кількість кроків генерації. Більше кроків можуть покращити якість, але збільшують час генерації.
    
    Returns:
    - image (PIL.Image.Image): Згенероване зображення.
    """
    try:
        # Токенізація промпту та перенесення на пристрій
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        
        # Генерація текстових ембеддінгів за допомогою обох енкодерів
        text_embeds1 = text_encoder(**inputs).last_hidden_state
        text_embeds2 = text_encoder_2(**inputs).last_hidden_state
        
        # Переконаємося, що ембеддінги також на пристрої
        text_embeds1 = text_embeds1.to(device)
        text_embeds2 = text_embeds2.to(device)
        
        # Генерація зображення з передачею текстових ембеддінгів
        output = pipe(
            prompt=prompt,
            text_embeds=(text_embeds1, text_embeds2),
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps
        )
        
        # Отримання першого згенерованого зображення
        image = output.images[0]
        return image
    except TypeError as e:
        print(f"Виникла помилка TypeError: {e}")
        print("Переконайтесь, що всі необхідні аргументи передані правильно і сумісні з конфігурацією пайплайна.")
        return None
    except Exception as e:
        print(f"Виникла несподівана помилка: {e}")
        return None

## Приклад Використання Функції для Генерації Зображення

In [ ]:
# Визначення промпту
prompt = "A serene sunset over the mountains with a lake in the foreground, ultra-realistic"

# Генерація зображення
generated_image = generate_image(prompt, guidance_scale=10.0, num_inference_steps=100)

# Відображення згенерованого зображення
if generated_image:
    plt.figure(figsize=(10, 10))
    plt.imshow(generated_image)
    plt.axis('off')
    plt.show()
else:
    print("Не вдалося згенерувати зображення.")

## Додаткові Рекомендації

1. **Переконайтесь, що Всі Компоненти на Одному Пристрої:**
   - Усі моделі та тензори повинні бути перенесені на один пристрій (`cuda` або `cpu`).
   - Використовуйте `device_map="auto"` при завантаженні пайплайна для автоматичного розподілення.

2. **Використовуйте Методи `.to(device)`:**
   - Переносьте кожен компонент (модель, енкодери) на потрібний пристрій вручну, якщо це необхідно.

3. **Перевірка Доступності GPU:**
   - Переконайтесь, що у вас є доступ до GPU та він правильно налаштований.

4. **Логування та Дебагінг:**
   - Додавайте додаткові `print` заяви для перевірки, на якому пристрої знаходяться тензори та моделі.

5. **Документація та Спільнота:**
   - Перегляньте [документацію Diffusers](https://huggingface.co/docs/diffusers/index) для додаткової інформації.
   - Звертайтесь до [спільноти Hugging Face](https://discuss.huggingface.co/) за допомогою, якщо проблема залишається.